In [ ]:
from pydrake.all import (
    ApplySimulatorConfig,
    SimulatorConfig,
    GetIntegrationSchemes,
)

In [ ]:
ApplySimulatorConfig

In [ ]:
SimulatorConfig().integration_scheme

In [ ]:
GetIntegrationSchemes()